In [37]:
import overpy
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np

def geocode_address(address):
    geolocator = Nominatim(user_agent="your_app_name")
    location = geolocator.geocode(address)
    return (location.latitude, location.longitude)


def get_nearby_amenities(address):
    try:
        coordinates = geocode_address(address)

        overpass_query = f"""
            node(around:500, {coordinates[0]}, {coordinates[1]})
            ["amenity"];
            out;
        """

        api = overpy.Overpass()
        result = api.query(overpass_query)

        amenities = [node.tags.get('amenity', 'N/A') for node in result.nodes]
        return ", ".join(amenities)
    
    except Exception as e:
        return str(e)

def process_excel(input_file, output_file):
    df = pd.read_excel(input_file)

    df['Amenities'] = df['Address'].apply(get_nearby_amenities)

    df.to_csv(output_file, index=False)

# Example usage
input_excel_file = "input_file.xlsx"
output_csv_file = "output_amenities.csv"
process_excel(input_excel_file, output_csv_file)


In [38]:
# Create data frame
csv = pd.read_csv("output_amenities.csv", encoding='utf-8')
amt = pd.DataFrame(csv)

# Extracting all categories from amenities 
amenities_mentioned = amt["Amenities"].str.split().explode().unique().tolist()

# Print list of unique amenities
print(amenities_mentioned)

['post_box,', 'recycling,', 'fast_food,', 'restaurant,', 'police,', 'bar,', 'cafe,', 'fire_station,', 'clinic,', 'bank,', 'bicycle_parking,', 'parking,', 'toilets,', 'lockers,', 'bench,', 'drinking_water,', 'bicycle_rental,', 'parking_entrance,', 'vending_machine,', 'piano,', 'charging_station,', 'doctors,', 'clock,', 'waste_basket,', 'boat_rental,', 'public_bookcase,', 'vending_machine', 'fuel,', 'motorcycle_parking,', 'bench', 'pharmacy,', 'pub,', 'waste_disposal,', 'ice_cream,', 'studio,', 'place_of_worship,', 'bbq,', 'taxi,', 'fountain,', 'dentist,', 'shelter,', 'arts_centre,', 'theatre,', 'kindergarten,', 'school,', 'food_court,', 'atm,', 'parking_entrance', 'townhall,', 'healthcare,', 'social_centre,', 'community_centre,', 'dental_hygienist,', 'social_facility,', 'cinema,', 'fountain', 'bureau_de_change,', 'bus_station,', 'nightclub,', 'casino,', 'coworking_space,', 'accupuncture,', 'brothel,', 'bicycle_repair_station,', 'cannabis,', 'car_rental,', 'ferry_terminal,', 'bicycle_par

In [39]:
excel = "google_maps_reviews_english_googletrans.xlsx"

# Read Excel file into DataFrame
rating_amenities = pd.read_excel(excel)

# Drop specific columns from the original DataFrame
rating_amenities = rating_amenities.drop(["name", "review_text", "location_link", "reviews_total", "review_datetime_utc", "reviews_per_score_1", "reviews_per_score_2", "reviews_per_score_3", "reviews_per_score_4", "reviews_per_score_5", "review_text_english_googletrans"], axis=1)

# Display new DataFrame
print(rating_amenities)


                         place_id  rating  review_rating
0     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.3              2
1     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.3              5
2     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.3              1
3     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.3              1
4     ChIJCwHQg38JxkcRJeVtM9mPCnM     3.3              5
...                           ...     ...            ...
1210  ChIJ5_ZVzPwzxEcRnORjwSzG0fU     4.6              5
1211  ChIJ5_ZVzPwzxEcRnORjwSzG0fU     4.6              5
1212  ChIJ5_ZVzPwzxEcRnORjwSzG0fU     4.6              5
1213  ChIJ5_ZVzPwzxEcRnORjwSzG0fU     4.6              5
1214  ChIJ5_ZVzPwzxEcRnORjwSzG0fU     4.6              5

[1215 rows x 3 columns]


In [40]:
#Count the ammenities
amenity_counts_dict = {f'{amenity}, count': [] for amenity in amenities_mentioned}

#Loop to count the amount of amenities per location
for amenities_str in amt["Amenities"]:
    for amenity in amenities_mentioned:
        count = amenities_str.count(amenity)
        
        amenity_counts_dict[f'{amenity}, count'].append(count)

#Formatting the naming of the columns
for amenity, counts in amenity_counts_dict.items():
    amt[amenity] = counts

amt["total_counts"] = amt[[amenity for amenity in amenity_counts_dict]].sum(axis=1)
amt.columns = amt.columns.str.replace(",,", ",")

# Display
display(amt)



/var/folders/95/3n7dglvx0g5g04d9d_rwdjq80000gn/T/ipykernel_2313/99377130.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amt[amenity] = counts
/var/folders/95/3n7dglvx0g5g04d9d_rwdjq80000gn/T/ipykernel_2313/99377130.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amt[amenity] = counts
/var/folders/95/3n7dglvx0g5g04d9d_rwdjq80000gn/T/ipykernel_2313/99377130.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider jo

,Address,Amenities,"post_box, count","recycling, count","fast_food, count","restaurant, count","police, count","bar, count","cafe, count","fire_station, count",...,"parking, count","events_venue, count","charging_station, count","ticket_validator, count","photo_booth, count","hospital, count","fuel, count","playground, count","balance_beam, count",total_counts
0,"Julianaplein 1, Amsterdam","post_box, recycling, fast_food, restaurant, re...",4,14,3,8,1,1,3,1,...,11,0,1,0,0,0,0,0,0,198
1,"Joan Muyskenweg 22, 1096 CJ Amsterdam","parking, post_box, parking, fuel, charging_sta...",1,0,0,0,0,0,0,0,...,3,0,5,0,0,0,1,0,0,32
2,"Nieuwe Leeuwarderweg, 1021 BZ Amsterdam","fuel, post_box, fast_food, pharmacy, pub, pub,...",2,17,6,5,0,0,1,0,...,6,0,1,0,0,0,1,0,0,227
3,"Spoorslag 29, 1082 MM Amsterdam","cafe, parking, place_of_worship, restaurant, c...",1,1,5,15,0,2,16,0,...,16,0,1,0,0,0,0,0,0,249
4,"Julianaplein Oost, 1097 DN Amsterdam","post_box, recycling, fast_food, restaurant, re...",4,15,4,8,1,1,3,1,...,11,0,2,0,0,0,0,0,0,206
5,"Termini 23, Amsterdam","townhall, healthcare, social_centre, dentist, ...",1,33,5,5,0,0,2,0,...,7,0,3,0,0,0,0,0,0,239
6,"Stationsplein 39, 1012 AB Amsterdam","restaurant, pub, pub, pub, restaurant, bureau_...",1,6,59,117,0,24,31,0,...,23,0,0,0,0,0,0,0,0,691
7,"Cornelis Lelylaan 35, 1062 HD Amsterdam","recycling, recycling, recycling, post_box, res...",3,29,5,3,0,0,3,0,...,6,0,0,0,0,0,0,0,0,167
8,"Europaboulevard 4A, 1083 AD Amsterdam","theatre, conference_centre, parking, parking, ...",1,1,0,3,0,0,1,0,...,13,0,0,0,0,0,0,0,0,163
9,"Kattenburgerstraat 5, 1018 JA Amsterdam","pub, post_box, restaurant, restaurant, restaur...",3,5,2,17,0,0,5,1,...,23,0,5,0,0,0,0,0,0,263


In [41]:
#add place ID
place_ID_df = pd.read_excel("INPUT_ALL_INCLUDED.xlsx")

#Merge DataFrames based on adress and place_id
merged_df = pd.merge(amt, place_ID_df, on = "Address", how = "left")

amt["place_id,"] = place_ID_df.groupby("Address")["place_id"].transform("first")

display(amt)

/var/folders/95/3n7dglvx0g5g04d9d_rwdjq80000gn/T/ipykernel_2313/1653105394.py:24: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  amt["place_id,"] = place_ID_df.groupby("Address")["place_id"].transform("first")


,Address,Amenities,"post_box, count","recycling, count","fast_food, count","restaurant, count","police, count","bar, count","cafe, count","fire_station, count",...,"events_venue, count","charging_station, count","ticket_validator, count","photo_booth, count","hospital, count","fuel, count","playground, count","balance_beam, count",total_counts,"place_id,"
0,"Julianaplein 1, Amsterdam","post_box, recycling, fast_food, restaurant, re...",4,14,3,8,1,1,3,1,...,0,1,0,0,0,0,0,0,198,NaN
1,"Joan Muyskenweg 22, 1096 CJ Amsterdam","parking, post_box, parking, fuel, charging_sta...",1,0,0,0,0,0,0,0,...,0,5,0,0,0,1,0,0,32,ChIJCwHQg38JxkcRJeVtM9mPCnM
2,"Nieuwe Leeuwarderweg, 1021 BZ Amsterdam","fuel, post_box, fast_food, pharmacy, pub, pub,...",2,17,6,5,0,0,1,0,...,0,1,0,0,0,1,0,0,227,ChIJxZHiajgLxkcRlOo3zmIibOs
3,"Spoorslag 29, 1082 MM Amsterdam","cafe, parking, place_of_worship, restaurant, c...",1,1,5,15,0,2,16,0,...,0,1,0,0,0,0,0,0,249,ChIJYyOpnFQIxkcRErWFhYVuAeQ
4,"Julianaplein Oost, 1097 DN Amsterdam","post_box, recycling, fast_food, restaurant, re...",4,15,4,8,1,1,3,1,...,0,2,0,0,0,0,0,0,206,ChIJ15K9igQKxkcR8A7le6H02hI
5,"Termini 23, Amsterdam","townhall, healthcare, social_centre, dentist, ...",1,33,5,5,0,0,2,0,...,0,3,0,0,0,0,0,0,239,ChIJPd2JhH8JxkcRQ8jSHUchci0
6,"Stationsplein 39, 1012 AB Amsterdam","restaurant, pub, pub, pub, restaurant, bureau_...",1,6,59,117,0,24,31,0,...,0,0,0,0,0,0,0,0,691,ChIJn0fdmPwJxkcRYe7a-6JHfpI
7,"Cornelis Lelylaan 35, 1062 HD Amsterdam","recycling, recycling, recycling, post_box, res...",3,29,5,3,0,0,3,0,...,0,0,0,0,0,0,0,0,167,ChIJf_z8xbcJxkcRCc-nQwIMAP8
8,"Europaboulevard 4A, 1083 AD Amsterdam","theatre, conference_centre, parking, parking, ...",1,1,0,3,0,0,1,0,...,0,0,0,0,0,0,0,0,163,ChIJ71V5XiPixUcRSKlH5OYy-iE
9,"Kattenburgerstraat 5, 1018 JA Amsterdam","pub, post_box, restaurant, restaurant, restaur...",3,5,2,17,0,0,5,1,...,0,5,0,0,0,0,0,0,263,ChIJLWMViCIKxkcRi1ThzDUCzCo


In [42]:
#merge the two to have ratings and amenities in one DataFrame
rating_and_count = pd.merge(amt, rating_amenities, on = "place_id", how = "left")

#turn index column into 
rating_and_count.reset_index(inplace = True)

#Display merged DataFrame
display(rating_and_count)

#Save to csv
rating_and_count.to_csv("rating_and_count.csv", index = False)


KeyError: 'place_id'